# Arco-IT GmbH MQP User Interface

## This user interface will allow the user to:

1. Import a data file to be analyzed (CSV)

    1a. Have the user select how many rows they wish to see
    
    1b. Have the user select which rows they would like to omit
2. Approve, Deny Specific Traffic
3. Generate Rules
4. Approve, Deny, or modify generated rules

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

#This is where the file upload button will be created
upload = widgets.FileUpload(
    description = 'Upload CSV',
    multiple = False,
    accept = '.csv'
)
#display the button
display(upload)


FileUpload(value={}, accept='.csv', description='Upload CSV')

In [3]:
#this takes the resulting dictionary and assigns it to a variable 
uploaded_file_val = upload.value
if(len(uploaded_file_val) == 0):
    raise Exception("You must input a file")
#show that variable
#uploaded_file_val

In [4]:
#access whats important
file_name = next(iter(uploaded_file_val))
#we get the first element in the dictionary
uploaded_file_content = uploaded_file_val[file_name]['content']
#uploaded_file_content

In [5]:
import codecs
#decodes from bytes to list and splits the data accordingly
data = uploaded_file_content.decode('utf-8').splitlines()
#below line removes quotes from decoding by iterating through list
data = [i.replace('"', ' ') for i in data]
#data

In [6]:
import csv
import re

#This creates a csv file within the directory that has the original data
with open("inventory_data.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+,',line))

In [8]:
#This gets the number of rows within the CSV
with open('inventory_data.csv') as counter:
    row_counter_with_header = sum(1 for line in counter)
row_counter_without_header = row_counter_with_header-1
#This is a slider to select how many rows would like to be displayed
slider_result = widgets.IntSlider(
    value=1,
    min=1,
    max=row_counter_without_header, #we have to subtract here to include the header
    step=1,
    description='Rows:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
slider_result

IntSlider(value=1, continuous_update=False, description='Rows:', max=1874, min=1)

In [13]:
#Outputting the CSV, range here is to display the header at all times
original_csv = pd.read_csv('inventory_data.csv', skiprows = range(1,(row_counter_with_header - slider_result.value)))
original_csv

,Address A,Port A,Address B,Port B,Packets,Bytes,Packets A -> B,Bytes A -> B,Packets B -> A,Bytes B -> A,Rel Start,Duration,Bits/s A -> B,Bits/s B -> A
0,192.168.100.2,60371,157.55.56.173,80,3,194,3,194,0,0,877.579045,8.999548,172.453105,0
1,192.168.100.2,60372,157.56.52.35,80,3,194,3,194,0,0,877.579046,8.999548,172.453105,0
2,192.168.100.2,60373,111.221.77.170,80,3,194,3,194,0,0,877.580154,8.998441,172.474321,0
3,192.168.100.2,60374,91.190.218.52,443,3,194,3,194,0,0,877.750186,8.999809,172.448104,0
4,192.168.100.2,60375,91.190.218.52,80,3,194,3,194,0,0,879.360044,8.999510,172.453834,0
5,192.168.100.2,60377,65.55.223.17,443,3,194,3,194,0,0,886.119864,9.000660,172.431799,0
6,192.168.100.2,60378,65.55.223.17,80,3,194,3,194,0,0,887.730104,9.000373,172.437298,0
7,192.168.100.2,60381,157.56.52.41,443,2,132,2,132,0,0,898.261471,2.970304,355.519166,0
8,192.168.100.2,60383,157.55.130.155,443,2,132,2,132,0,0,899.291877,2.970637,355.479313,0
9,192.168.100.2,60387,157.56.52.41,80,2,132,2,132,0,0,899.851871,3.000662,351.922343,0


In [14]:
#This part is going to have the user enter which rows they would like to remove
print("In the following textbox, input the rows that you would like to omit from the above dataset. Follows zero-based indexxing. Meaning first row is 0, second is 1, and so on.")
deny_list_input = widgets.Textarea(placeholder = "Separate the row values with commas",
                        description = "Rows: ",
                        disabled = False,
                        layout = widgets.Layout(width='auto', height='40px')#auto adjusts size)
deny_list_input

In the following textbox, input the rows that you would like to omit from the above dataset. Follows zero-based indexxing. Meaning first row is 0, second is 1, and so on.


Textarea(value='', description='Rows: ', layout=Layout(height='40px', width='auto'), placeholder='Separate the…

In [15]:
#separate variables
deny_list = deny_list_input.value.split(",")
#convert into integers
deny_list_ints = [int(i) for i in deny_list]
#deny_list_ints #this just shows what the contents are


In [19]:
#drop the selected rows from the dataset
original_csv.drop(original_csv.index[deny_list_ints], inplace = True)
original_csv


,Address A,Port A,Address B,Port B,Packets,Bytes,Packets A -> B,Bytes A -> B,Packets B -> A,Bytes B -> A,Rel Start,Duration,Bits/s A -> B,Bits/s B -> A
0,192.168.100.2,60371,157.55.56.173,80,3,194,3,194,0,0,877.579045,8.999548,172.453105,0
4,192.168.100.2,60375,91.190.218.52,80,3,194,3,194,0,0,879.360044,8.999510,172.453834,0
6,192.168.100.2,60378,65.55.223.17,80,3,194,3,194,0,0,887.730104,9.000373,172.437298,0
8,192.168.100.2,60383,157.55.130.155,443,2,132,2,132,0,0,899.291877,2.970637,355.479313,0
10,192.168.100.2,60388,111.221.77.162,443,1,66,1,66,0,0,900.302073,0.000000,0.000000,0
11,192.168.100.2,60389,157.55.130.155,80,1,66,1,66,0,0,900.881977,0.000000,0.000000,0
12,192.168.100.2,60390,157.55.235.153,443,1,66,1,66,0,0,901.342021,0.000000,0.000000,0
13,192.168.100.2,60391,157.55.56.161,443,1,66,1,66,0,0,901.342023,0.000000,0.000000,0
14,192.168.100.2,60392,157.55.130.146,443,1,66,1,66,0,0,901.342025,0.000000,0.000000,0
15,192.168.100.2,60393,111.221.77.162,80,1,66,1,66,0,0,901.913229,0.000000,0.000000,0


In [20]:
#This creates another csv file within the directory that has the original data
with open("inventory_data.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+,',line))

In [21]:
#creating the export rules button
export_button = widgets.Button(description= "Export Rules")
export_button

Button(description='Export Rules', style=ButtonStyle())